# Mutations for differential evolution revised

In this paper, core operation for Differential Evolution is researched, mutation. For DE, different types of mutation can easily change a chance to susccess for any function, as it strongly defines behavior of heuristics. With non-random cahnges to algorithm via mutation, it can be adapted to single-modal functions with strees to exploitation of current best solution or to multi-modal functions, encouraging exploration over exploitation through more random-wise mutations.

Several widely-used mutations are presented and its advantages and disadvantages are described with example of finding solution to one of De Jong testing functions - single-modal Rosenbrock 2D "banana" fuction. It is expected, that mutations using best candidates will be more successful than the one with random search, as the heuristic can not stuck in local minimum, for the Rosenbrock fucntion has only one, which happens to be global minimum as well.

In [1]:
import sys
import os
pwd = %pwd
sys.path.append(os.path.join(pwd, '../../src'))

# Ensure modules are reloaded on any change (very useful when developing code on the fly)
get_ipython().magic('load_ext autoreload')
get_ipython().magic('autoreload 2')

In [172]:
# Import external libraries
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook

### Evaluation

Heuristics results will be evaluated with standard criteria:

* Reliability (REL) - statistical probability of successful run
* Mean Number of Objective Function Evaluations (MNE)
* Feoktistov criterion (FEO) - MNE/REL

In [3]:
# Define criteria for heuristics evaluation
def rel(x):
    return len([n for n in x if n < np.inf])/len(x)
def mne(x):
    return np.mean([n for n in x if n < np.inf])
def feo(x):
    return mne(x)/rel(x)

In [4]:
# Import objective function and heuristics dependencies
from objfun import Rosenbrock
from heur import DifferentialEvolution
from de_mutations import deRand, deBest, deRand2, deBest2, deCurrentToBest1, deRandToBest1

In [5]:
NUM_RUNS = 100
maxeval = 1000

### Rosenbrock funciton

For testing purposes - different tuning of parameters for given mutations, Rosenbrock function will be used. As the parameters tuning is question of each function it self, there is no certainity, that the same set of paramteres for given mutation will be os use for different function (i.e. No Free Lunch theorem).

Definition of function described here (http://www.geatbx.com/docu/fcnindex-01.html#P129_5426) will be used.

In [6]:
rosenbrock2 = Rosenbrock(n=2, eps=0.1)
fstar = rosenbrock2.evaluate(np.ones(2))

### DE parameters

For Differential evolution three parameteres need to be supplied, all of the are very important setting for all the mutations revised in this notebook.

* N: population size, to properly test all the mutations, has to be $> 5$ (some mutations need bigger amont of individuals),
* CR: crossover probability,
* F: differential weight, has to be in $[0,2]$.

### Mutations in short

Implemented mutaions are the ones originally presented by publicators of DE Price and Storn. This subset of known mutations contains both explorative (Rand1,Rand2) as exhaustive (Best1,Best2) approaches and in addition some others, trying to combine both. There is always trade-off between exploration and exhaustion, which has to be handled accordingly and mutations has to be tailored to the needs of objective function (i.e. if the function is multimodal, different mutation has to be used - in favor of exploration - than in case of unimodal.) Following brief list contains all the implemented mutations and their definition.

Numbers $r_1, \ldots, r_5$ are random numbers from range $1, \ldots, N$. In all cases, $x_{current} \ne x_{r_1} \ne \ldots \ne x_{r_5} \ne x_{best}$

* **Rand1** - simplest of random mutaions, $x_{new} = x_{r_1} + F (x_{r_2} - x_{r_3})$.

* **Rand2** - similar to deRand, uses more individuals, hence the need for higher value of $N$, $x_{new} = x_{r_1} + F (x_{r_2} - x_{r_3}) + F (x_{r_4} - x_{r_5})$.

* **Best1** - computationally harder, espetially for bigger populations, because of the search for the best individual in population, denoted $x_{best}$, $x_{new} = x_{best} + F (x_{r_1} - x_{r_2})$.

* **Best2** - similar to deBest, uses more individials, $x_{new} = x_{best} + F (x_{r_1} - x_{r_2})+ F (x_{r_3} - x_{r_4})$.

* **Current to Best** - $x_{new} = x_{current} + F (x_{best} - x_{current})+ F (x_{r_1} - x_{r_2})$.

* **Rand to Best** - $x_{new} = x_{r_1} + F (x_{best} - x_{r_1})+ F (x_{r_2} - x_{r_3})$.

### Experiment setup

In [7]:
def experiment_de(of, maxeval, num_runs, N, CR, F, mutation):
    results = []
    for i in tqdm_notebook(range(num_runs)):
        result = DifferentialEvolution(of, maxeval=maxeval, N=N, CR=CR, F=F, mutation=mutation['algorithm']).search()
        result['run'] = i
        result['heur'] = 'DE_{}_{}_{}_{}'.format(N, CR, F, mutation['name'])
        results.append(result)
    return pd.DataFrame(results, columns=['heur', 'run','best_x', 'best_y', 'neval'])

### Initial set of parameters
Peek at the results with some initial set of parameters.

* Increase in population size gives better chance to find good solution, but is only useful for mutations with search for the best individual, which are deBest, deBest2, deCurrentToBest1 and deRandToBest1.

* Increase in F puts more emphasis on the mutation itself and increase in CR raises a chance of mutation.

In [13]:
def initMutations(F, N, CR):
    mutations = [
    {'algorithm': deRand(F=F, N=N, CR=CR), 'name': 'deRand1'},
    {'algorithm': deBest(F=F, N=N, CR=CR), 'name': 'deBest1'},
    {'algorithm': deRand2(F=F, N=N, CR=CR), 'name': 'deRand2'},
    {'algorithm': deBest2(F=F, N=N, CR=CR), 'name': 'deBest2'},
    {'algorithm': deCurrentToBest1(F=F, N=N, CR=CR), 'name': 'deCurrentToBest1'},
    {'algorithm': deRandToBest1(F=F, N=N, CR=CR), 'name': 'deRandToBest1'},
    ]
    return mutations

In [20]:
F=1
N=6
CR=0.5

In [21]:
mutations = initMutations(F, N, CR)

In [22]:
de_results = pd.DataFrame()
for mutation in mutations:
    de_res = experiment_de(of=rosenbrock2, maxeval=maxeval, num_runs=NUM_RUNS, N=N, CR=CR, F=F, mutation=mutation)
    de_results = pd.concat([de_results, de_res], axis=0)

In [23]:

de_results_pivot = de_results.pivot_table(
    index=['heur'],
    values=['neval'],
    aggfunc=(rel, mne, feo)
)['neval']
de_results_pivot = de_results_pivot.reset_index()

de_results_pivot

,heur,rel,mne,feo
0,DE_6_0.5_1_deBest1,0.27,104.148148,385.733882
1,DE_6_0.5_1_deBest2,0.03,102.333333,3411.111111
2,DE_6_0.5_1_deCurrentToBest1,0.32,163.000000,509.375000
3,DE_6_0.5_1_deRand1,0.43,144.860465,336.884803
4,DE_6_0.5_1_deRand2,0.55,339.018182,616.396694
5,DE_6_0.5_1_deRandToBest1,0.38,159.473684,419.667590


From the first test, it can be seen, that rand1 mutation was best. It can be said, the mutations with selection of best individual was worse (best1 and best2). The reason could be small population, with the increase in population, these mutations could yield better results.

In [31]:
F=1
N=12
CR=0.5

In [32]:
mutations = initMutations(F, N, CR)

In [33]:
de_results = pd.DataFrame()
for mutation in mutations:
    de_res = experiment_de(of=rosenbrock2, maxeval=maxeval, num_runs=NUM_RUNS, N=N, CR=CR, F=F, mutation=mutation)
    de_results = pd.concat([de_results, de_res], axis=0)

In [34]:

de_results_pivot = de_results.pivot_table(
    index=['heur'],
    values=['neval'],
    aggfunc=(rel, mne, feo)
)['neval']
de_results_pivot = de_results_pivot.reset_index()

de_results_pivot

,heur,rel,mne,feo
0,DE_12_0.5_1_deBest1,0.85,296.341176,348.636678
1,DE_12_0.5_1_deBest2,0.71,489.690141,689.704424
2,DE_12_0.5_1_deCurrentToBest1,0.91,381.318681,419.031518
3,DE_12_0.5_1_deRand1,0.98,348.734694,355.851728
4,DE_12_0.5_1_deRand2,0.78,437.205128,560.519395
5,DE_12_0.5_1_deRandToBest1,0.90,404.122222,449.024691


The reliability of all the experiments rose with growth in population, most significantly for best1 and best2. According to Feocristov criterium, best1 was the best for this run, due to small number of evaluation in opposition to rand1, which has high reliability, but also needs more evaluations.

Lets see, how increase of desired precision of outcome affect these results.

In [39]:
rosenbrock2 = Rosenbrock(n=2, eps=0.001)

In [40]:
de_results = pd.DataFrame()
for mutation in mutations:
    de_res = experiment_de(of=rosenbrock2, maxeval=maxeval, num_runs=NUM_RUNS, N=N, CR=CR, F=F, mutation=mutation)
    de_results = pd.concat([de_results, de_res], axis=0)

In [41]:

de_results_pivot = de_results.pivot_table(
    index=['heur'],
    values=['neval'],
    aggfunc=(rel, mne, feo)
)['neval']
de_results_pivot = de_results_pivot.reset_index()

de_results_pivot

,heur,rel,mne,feo
0,DE_12_0.5_1_deBest1,0.26,757.692308,2914.201183
1,DE_12_0.5_1_deBest2,0.01,286.000000,28600.000000
2,DE_12_0.5_1_deCurrentToBest1,0.13,713.692308,5489.940828
3,DE_12_0.5_1_deRand1,0.39,792.102564,2031.032216
4,DE_12_0.5_1_deRand2,0.02,313.500000,15675.000000
5,DE_12_0.5_1_deRandToBest1,0.09,865.555556,9617.283951


Most of the experiments did not finish, because maximal number of evaluations was exhausted. It is logical to increase the maximum if more precise solution is to be found.

In [173]:
NUM_RUNS = 100
maxeval = 10000

In [174]:
de_results = pd.DataFrame()
for mutation in mutations:
    de_res = experiment_de(of=rosenbrock2, maxeval=maxeval, num_runs=NUM_RUNS, N=N, CR=CR, F=F, mutation=mutation)
    de_results = pd.concat([de_results, de_res], axis=0)

In [46]:
de_results_pivot = de_results.pivot_table(
    index=['heur'],
    values=['neval'],
    aggfunc=(rel, mne, feo)
)['neval']
de_results_pivot = de_results_pivot.reset_index()

de_results_pivot

,heur,rel,mne,feo
0,DE_12_0.5_1_deBest1,0.44,1153.477273,2621.539256
1,DE_12_0.5_1_deBest2,1.00,2205.060000,2205.060000
2,DE_12_0.5_1_deCurrentToBest1,0.44,1152.931818,2620.299587
3,DE_12_0.5_1_deRand1,0.82,1112.695122,1356.945271
4,DE_12_0.5_1_deRand2,1.00,2380.890000,2380.890000
5,DE_12_0.5_1_deRandToBest1,0.55,1414.072727,2571.041322


Best2 and rand2 are the most reliable, but also takes twice as much evaluations and are therefore slower. To Feocristov, rand1 is far best. Lets lower the precision and tweak N, F and CR parameters for the mutations.

In [62]:
maxeval = 1000
rosenbrock2 = Rosenbrock(n=2, eps=0.1)

### Random-wise mutations

In [63]:
def initRandMutations(F, N, CR):
    mutations = [
    {'algorithm': deRand(F=F, N=N, CR=CR), 'name': 'deRand1'},
    {'algorithm': deRand2(F=F, N=N, CR=CR), 'name': 'deRand2'},
    ]
    return mutations

In [90]:
F=0.3
N=20
CR=0.9
mutations = initRandMutations(F, N, CR)

In [91]:
de_results = pd.DataFrame()
for mutation in mutations:
    de_res = experiment_de(of=rosenbrock2, maxeval=maxeval, num_runs=NUM_RUNS, N=N, CR=CR, F=F, mutation=mutation)
    de_results = pd.concat([de_results, de_res], axis=0)

In [92]:
de_results_pivot = de_results.pivot_table(
    index=['heur'],
    values=['neval'],
    aggfunc=(rel, mne, feo)
)['neval']
de_results_pivot = de_results_pivot.reset_index()

de_results_pivot

,heur,rel,mne,feo
0,DE_20_0.9_0.3_deRand1,0.81,141.469136,174.653254
1,DE_20_0.9_0.3_deRand2,0.92,172.500000,187.500000


After few experiments, this setup seems to be good. With increase of $F$, reliability growths, but number of evaluations growths as well, keeping Feocristov criterium at approximately the same level. It can be seen, that rand2 method is more reliable than rand1, but it takes more evaluations, so there is tradeoff between reliability and time consumption.

### Best-finding mutations

In [93]:
def initBestMutations(F, N, CR):
    mutations = [
    {'algorithm': deBest(F=F, N=N, CR=CR), 'name': 'deBest1'},
    {'algorithm': deBest2(F=F, N=N, CR=CR), 'name': 'deBest2'},
    {'algorithm': deCurrentToBest1(F=F, N=N, CR=CR), 'name': 'deCurrentToBest1'},
    {'algorithm': deRandToBest1(F=F, N=N, CR=CR), 'name': 'deRandToBest1'},
    ]
    return mutations

It is my expectation, that these mutations will profit from broad population and lower value of $CR$, as it is desired to keep the best solution "on its way".

Lets try the same setup as for the random-wise mutations:

In [97]:
F=0.3
N=20
CR=0.9
mutations = initBestMutations(F, N, CR)

In [95]:
de_results = pd.DataFrame()
for mutation in mutations:
    de_res = experiment_de(of=rosenbrock2, maxeval=maxeval, num_runs=NUM_RUNS, N=N, CR=CR, F=F, mutation=mutation)
    de_results = pd.concat([de_results, de_res], axis=0)

In [96]:
de_results_pivot = de_results.pivot_table(
    index=['heur'],
    values=['neval'],
    aggfunc=(rel, mne, feo)
)['neval']
de_results_pivot = de_results_pivot.reset_index()

de_results_pivot

,heur,rel,mne,feo
0,DE_20_0.9_0.3_deBest1,0.86,296.325581,344.564630
1,DE_20_0.9_0.3_deBest2,1.00,121.400000,121.400000
2,DE_20_0.9_0.3_deCurrentToBest1,0.47,194.276596,413.354459
3,DE_20_0.9_0.3_deRandToBest1,0.60,115.000000,191.666667


Best2 is far superior so far, and after other parameters setup, no better result could be achieved with best1 and best2. With growth of population size, reliability grows, but number of evaluations grows more rapidli thus lowering feocristov criterium. Let us now examine currentToBest1 and randToBest1 mutations

In [116]:
def initLastMutations(F, N, CR):
    mutations = [
    {'algorithm': deCurrentToBest1(F=F, N=N, CR=CR), 'name': 'deCurrentToBest1'},
    {'algorithm': deRandToBest1(F=F, N=N, CR=CR), 'name': 'deRandToBest1'},
    ]
    return mutations

In [146]:
F=1.0
N=20
CR=0.97
mutations = initLastMutations(F, N, CR)

In [147]:
de_results = pd.DataFrame()
for mutation in mutations:
    de_res = experiment_de(of=rosenbrock2, maxeval=maxeval, num_runs=NUM_RUNS, N=N, CR=CR, F=F, mutation=mutation)
    de_results = pd.concat([de_results, de_res], axis=0)

In [148]:
de_results_pivot = de_results.pivot_table(
    index=['heur'],
    values=['neval'],
    aggfunc=(rel, mne, feo)
)['neval']
de_results_pivot = de_results_pivot.reset_index()

de_results_pivot

,heur,rel,mne,feo
0,DE_20_0.97_1.0_deCurrentToBest1,0.99,284.373737,287.246199
1,DE_20_0.97_1.0_deRandToBest1,0.97,363.103093,374.333085


These experiments are strongly reliable for $CR$ value close to $1$ and $F$ value deviationg around $1$, which is usable property, but number of evaluations is much higher than with best2 or rand1/2. Lowering F value leads to satisfyingly small number of evaluations but with the cost of low reliability and lowering CR leads to great reliability but higher number of evaluations (see below), so this setup is balanced.

In [158]:
F=1
N=20
CR=0.5
mutations = initLastMutations(F, N, CR)

In [159]:
de_results = pd.DataFrame()
for mutation in mutations:
    de_res = experiment_de(of=rosenbrock2, maxeval=maxeval, num_runs=NUM_RUNS, N=N, CR=CR, F=F, mutation=mutation)
    de_results = pd.concat([de_results, de_res], axis=0)

In [160]:
de_results_pivot = de_results.pivot_table(
    index=['heur'],
    values=['neval'],
    aggfunc=(rel, mne, feo)
)['neval']
de_results_pivot = de_results_pivot.reset_index()

de_results_pivot

,heur,rel,mne,feo
0,DE_20_0.5_1_deCurrentToBest1,0.99,503.747475,508.835833
1,DE_20_0.5_1_deRandToBest1,0.99,498.070707,503.101724


### Best setups
This table shows results for the best setups of parameters for the mutations revised

In [162]:
de_results_rand = pd.DataFrame()
F=0.3
N=20
CR=0.9
mutations = initRandMutations(F, N, CR)
for mutation in mutations:
    de_res = experiment_de(of=rosenbrock2, maxeval=maxeval, num_runs=NUM_RUNS, N=N, CR=CR, F=F, mutation=mutation)
    de_results_rand = pd.concat([de_results_rand, de_res], axis=0)

In [163]:
de_results_best= pd.DataFrame()
F=0.3
N=20
CR=0.9
mutations = initBestMutations(F, N, CR)
for mutation in mutations:
    de_res = experiment_de(of=rosenbrock2, maxeval=maxeval, num_runs=NUM_RUNS, N=N, CR=CR, F=F, mutation=mutation)
    de_results_best = pd.concat([de_results_best, de_res], axis=0)

In [164]:
de_results_last= pd.DataFrame()
F=1.0
N=20
CR=0.97
mutations = initLastMutations(F, N, CR)
for mutation in mutations:
    de_res = experiment_de(of=rosenbrock2, maxeval=maxeval, num_runs=NUM_RUNS, N=N, CR=CR, F=F, mutation=mutation)
    de_results_last = pd.concat([de_results_last, de_res], axis=0)

In [171]:
de_results_pivot_final = pd.concat([de_results_rand,de_results_best,de_results_last], axis=0)
de_results_pivot_final = de_results_final.pivot_table(
    index=['heur'],
    values=['neval'],
    aggfunc=(rel, mne, feo)
)['neval']
de_results_pivot_final = de_results_pivot_final.reset_index()

de_results_pivot_final.sort_values('feo')

,heur,rel,mne,feo
3,DE_20_0.9_0.3_deBest2,1.00,156.490000,156.490000
5,DE_20_0.9_0.3_deRand1,0.86,143.046512,166.333153
7,DE_20_0.9_0.3_deRandToBest1,0.47,94.000000,200.000000
6,DE_20_0.9_0.3_deRand2,0.86,204.732558,238.061114
2,DE_20_0.9_0.3_deBest1,0.88,313.613636,356.379132
1,DE_20_0.97_1.0_deRandToBest1,0.99,365.949495,369.645954
0,DE_20_0.97_1.0_deCurrentToBest1,0.96,382.750000,398.697917
4,DE_20_0.9_0.3_deCurrentToBest1,0.59,284.440678,482.102844


## Conclusion
The goal of this notebook was not to find the best parameters setup for optimizing Rosenbrock function with Differential Evolution, but to tweak the parameters for each of implemented mutaion. Some assumptions about influence of parameters was made base od the knowledge of the heuristics and some of the parameter tunig was made with aposterior information by experimenting with different setups.

Rosenbrock function can be characterized more as simple task than hard, because it is unimodal. This unimodality could favor methods using best individual in population when mutating, beacause there is no risk of stucking in local minimum. It showed up, that best mutation to be used is truly based on finding best individual. It also showed up, that totally random approach is very usefull, beacause classical random mutation was fast, reliable and with small number of evaluations. If there is need to keep machine time to the lowest, classical random mutation (rand1) should be used, as it was approximately twice as fast as best2, but if time consumption is not so much important, than the mutation is best2 with great reliability and low number of evaluations.

Other methods showed to be mediocore for this function, but I can imagine for example randToBest1 with $F = 1$ might be choice for harder tasks with big need for reliability, if there is use for time-consuming finding of best solution.

In order of NO FREE LUNCH theorem, this notebook does not show the best parameter setup for DE, but demonstrates some ways of using different mutations for DE on problem of optimization of unimodal 2D function with nontrivial solution.